In [661]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [662]:
from selenium import webdriver
from selenium.common import exceptions
import re
import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import time

# 크롤링

In [663]:
OUTPUT_FILE_NAME = 'output.txt'

In [664]:
wd = "C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver.exe"

In [665]:
import urllib.request
def get_text(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = ''
    for item in soup.find_all('div', id='realArtcContents'):
        text = text + str(item.find_all(text=True))
    return text

In [666]:
def clean_text(text):
    cleaned_text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\.]+", " ", str(text))
    return cleaned_text

In [667]:
def get_title(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'html.parser', from_encoding='utf-8')
    text = soup.find('h3',class_='articleSubecjt')
    return text

In [668]:
def nate_comment(url):
    driver = webdriver.Chrome(wd)
    url_nums = re.findall("\d+", url)
    addr = 'http://comm.news.nate.com/Comment/ArticleComment/List?artc_sq='+url_nums[0]+'n'+url_nums[1] # 댓글창 주소
    driver.get(addr)
    pages = 2
    try:
        for i in range(5):
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"body > div.commentBox.reply_hide > div:nth-child(3) > div.paging_wrap > a:nth-child("+str(pages)+")"))).click()
            time.sleep(2)
            print(pages, end=" ")
            pages+=1
    except exceptions.ElementNotVisibleException as e: # 페이지 끝
        pass
    except Exception as e: # 다른 예외 발생시 확인
        print(e)
    html = driver.page_source
    dom = BeautifulSoup(html, "lxml")
    # 댓글이 들어있는 페이지 전체 크롤링
    comments_raw = dom.find_all('dd',{'class':'usertxt'})
    # 댓글의 text만 뽑는다.
    comments = [comment.text.strip() for comment in comments_raw]
    return comments

In [669]:
def naver_comment(url):
    wd = "C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver"
    driver = webdriver.Chrome(wd)
    driver.get(url)
    pages = 0
    try:
        while True:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#cbox_module > div > div.u_cbox_paginate > a"))).click()
            time.sleep(1.5)
            print(pages, end=" ")
            pages+=1
    except exceptions.ElementNotVisibleException as e: # 페이지 끝
        pass
    
    except Exception as e: # 다른 예외 발생시 확인
        print(e)
    html = driver.page_source
    dom = BeautifulSoup(html, "lxml")
    # 댓글이 들어있는 페이지 전체 크롤링
    comments_raw = dom.find_all("span", {"class" : "u_cbox_contents"})
    # 댓글의 text만 뽑는다.
    comments = [comment.text for comment in comments_raw]
    return comments

# 키워드 추출

In [670]:
from krwordrank.sentence import summarize_with_sentences
def keyword_word(text):#키워드 단어 추출
    data = text.split('.')
    keywords, sents = summarize_with_sentences(data, num_keywords=10, num_keysents=10)
    return keywords

In [671]:
def keyword_sentence(url):#키워드 문장 추출
    m_text = get_text(url)
    r_text = clean_text(m_text)
    data = r_text.split('.')
    keywords, sents = summarize_with_sentences(data, num_keywords=10, num_keysents=10)
    return sents

# 모델 로딩

In [672]:
from tensorflow.keras.models import load_model
from konlpy.tag import Okt  
okt = Okt()

In [673]:
def model_load(comments):
    token_sent=[]
    for comment in comments:
        temp = okt.morphs(comment, stem=True)
        token_sent.append(temp)
    max_words = 35000
    tokenizer = Tokenizer(num_words = max_words) # 상위 35,000개의 단어만 보존
    tokenizer.fit_on_texts(token_sent) 
    token_sent = tokenizer.texts_to_sequences(token_sent)
    word_to_index = tokenizer.word_index
    vocab_size = len(word_to_index)+1
    max_len = 124
    X_data = pad_sequences(token_sent, maxlen=max_len)
    model = load_model('model8n.h5')
    predict = model.predict_classes(X_data)
    a = [1 for _ in range(len(X_data))]
    for i in range(len(X_data)):
        if(predict[i] == 0):
            a[i]=0
    return a

# 댓글 수치 판단

In [674]:
def neg_pos(results):
    pos = 0
    neg = 0
    for result in results:
        if result == 0:
            pos += 1
        else:
            neg += 1
    result = pos / (pos + neg) * 100
    return result

In [675]:
def clean(results):
    for result in results:
        if result == 1:
            result == 'bad comment'
    return result

# 악플 개수 세기

In [676]:
def count(num): #count는 악플 개수, number은 악플 위치
    count = 0
    number = []
    for i in range(1, len(num)):
        if num[i] == 1:
            count += 1
            number.append(i)
    return count, number

# 악플 데이터 삭제

In [677]:
def de_comment(comments, number):
    new = np.delete(comments, number)
    return new

# 악플 데이터 남기기

In [678]:
def count2(num): #count는 악플 개수, number은 악플 위치
    number = []
    for i in range(1, len(num)):
        if num[i] == 0:
            number.append(i)
    return number

In [679]:
def de_comment2(comments, number):
    new = np.delete(comments, number)
    return new

# 선플 데이터 넣기

# 모델 추가 train

In [680]:
import pandas as pd
from konlpy.tag import Okt
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

def train(comments, model_result):
    comment_data = pd.DataFrame(comments)
    comment_data.columns = ['comment']
    in_result = np.insert(model_result, 0, 0)
    de_result = np.delete(in_result,0)
    de_data = pd.DataFrame(de_result)
    de_data.columns = ['label']
    df_new = pd.concat([de_data, comment_data], axis = 1)
    X_data = df_new['comment']
    y_data = df_new['label']
    normalized_text = []
    for string in X_data.tolist():
        try:
            tokens = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]+", " ", string.lower())
        except Exception as e:
            break
    normalized_text.append(tokens)
    #df_new.comment = normalized_text
    okt = Okt()
    X_token=[]
    for sentence in df_new['comment']:
        temp_X = []
        temp_X = okt.morphs(sentence, stem=True) # 토큰화
        X_token.append(temp_X)
    max_words = 10000
    tokenizer = Tokenizer(num_words = max_words) # 상위 35,000개의 단어만 보존
    tokenizer.fit_on_texts(X_token) 
    X_token = tokenizer.texts_to_sequences(X_token)
    word_to_index = tokenizer.word_index
    vocab_size = len(word_to_index) + 1
    max_len = 124
    X_data = pad_sequences(X_token, maxlen=max_len)
    y_data = np.array(y_data).reshape(-1, 1)
    model = load_model('model8n.h5')
    history = model.fit(X_data, y_data, epochs=5, batch_size=32)

# 플라스크

In [681]:
from flask import Flask, render_template
from flask import request, send_file, url_for
from wtforms import Form, TextAreaField, validators, SelectField
import pickle
import sqlite3
import os

In [682]:
app = Flask(__name__)
url = ''

In [683]:
class ArticleForm(Form):
    site = SelectField('',choices=[(1,'NATE'),(2,'NAVER')])
    articleurl = TextAreaField('',[validators.DataRequired()])

In [684]:
@app.route('/')
def index():
    form = ArticleForm()
    return render_template('article_app.html', form=form)

In [685]:
@app.route('/title', methods = ['POST'])
def article():
    form = ArticleForm(request.form)
    if request.method == 'POST' and form.validate():
        global url
        url = request.form['articleurl']
        title = get_title(url)
        r_title = clean_text(title)
        m_text = get_text(url)
        r_text = clean_text(m_text)
        return render_template('title.html', title=r_title, content=r_text)
    return render_template('article_app.html', form=form)

In [686]:
@app.route('/result')
def review():
    global url
    cr_comment = nate_comment(url)
    '''if site == 'NATE':
        cr_comment = nate_comment(url)
    elif site == 'NAVER':
        cr_comment = naver_comment(url)'''
    results = model_load(cr_comment)
    cm_count, number = count(results)
    new_comment = de_comment(cr_comment, number)
    result = neg_pos(results) 
    return render_template('result.html', list2=new_comment, list4=result)

In [687]:
@app.route('/bad_comment')
def bad_comment():
    global url
    cr_comment = nate_comment(url)
    '''if site == 'NATE':
        cr_comment = nate_comment(url)
    elif site == 'NAVER':
        cr_comment = naver_comment(url)'''
    results = model_load(cr_comment)
    number = count2(results)
    new_comment = de_comment2(cr_comment, number)
    return render_template('bad_comment.html', list1 = new_comment)

In [688]:
@app.route('/no_train')
def no_train():
    return render_template('no_train.html')

In [689]:
@app.route('/thanks')
def feedback():
    global url
    cr_comment = nate_comment(url)
    '''if site == 'NATE':
        cr_comment = nate_comment(url)
    elif site == 'NAVER':
        cr_comment = naver_comment(url)'''
    results = model_load(cr_comment)
    train_check = train(cr_comment, results)
    return render_template('thanks.html')

In [660]:
if __name__ == '__main__':
    app.run()

* Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Apr/2020 07:36:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2020 07:36:36] "GET /static/reset.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2020 07:36:36] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2020 07:36:37] "GET /nate.png HTTP/1.1" 404 -
127.0.0.1 - - [02/Apr/2020 07:36:37] "GET /naver.png HTTP/1.1" 404 -
127.0.0.1 - - [02/Apr/2020 07:36:37] "GET /daum.png HTTP/1.1" 404 -
127.0.0.1 - - [02/Apr/2020 07:36:38] "GET /favicon.ico HTTP/1.1" 404 -
